## Setup

說明：這部分如果出現需要重新啟動的報錯 (You must restart the runtime in order to use newly installed versions.)，重新啟動就可以了(這個 cell 可能需要跑兩次)

In [ ]:
! pip install -U pip setuptools
! pip install concrete-ml transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.6.3
    Uninstalling setuptools-65.6.3:
      Successfully uninstalled setuptools-65.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
concrete-ml 1.0.2 requires setuptools==65.6.3, but you have setuptools 67.8.0 which is incompatible.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have to

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached setuptools-65.6.3-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.8.0
    Uninstalling setuptools-67.8.0:
      Successfully uninstalled setuptools-67.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.


## Prepare

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm
import random
from collections import Counter
from IPython.display import display


import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, AutoModel
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold

In [ ]:
device = torch.device('cuda', 0) if torch.cuda.is_available() else 'cpu'
print(device)

# BERT Tokenizer: 使用 'bert-base-cased' 版本
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

# BERT Model: 使用 'bert-base-cased' 版本
bert = AutoModel.from_pretrained('bert-base-cased').to(device)

cuda:0


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# config
config = {"embedding_path": "embedding_tensor.pt",
          "seed": 168,
          "learning_rate": 3e-4,
          "batch_size": 64,
          "epochs": 300}

In [ ]:
def set_random_seed(seed, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)

set_random_seed(config["seed"])

## Data preparation

In [ ]:
from numpy.lib.shape_base import apply_along_axis
# fetch the dataset using scikit-learn
import sklearn
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)
all_data = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 11314
size of validation set: 7532
classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
print(y_train)

[7 4 4 ... 3 1 8]


## BERT Embedding

In [ ]:
def get_embedding_tensor(context_ary):

    embedding_tensor = torch.ones((context_ary.shape[0], 768))


    for i in range(context_ary.shape[0]):

        context = context_ary[i]
        bert_input = tokenizer(context, padding='max_length', max_length=512,
                               truncation=True, return_tensors="pt")


        attention_mask = bert_input['attention_mask'].to(device)
        input_id = bert_input['input_ids'].squeeze(1).to(device)
        final_inputs = {'input_ids': input_id, 'attention_mask': attention_mask}
        outputs = bert(**final_inputs)

        ## Only the embedding vector of [CLS] is taken
        pooler_output = outputs.last_hidden_state[0][0].reshape(768)
        embedding_tensor[i] = pooler_output.detach().cpu()

        gc.collect()
        torch.cuda.empty_cache()

    return embedding_tensor

In [ ]:
# 計算每個樣本各自的 BERT Embedding
train_context_ary = np.array(x_train)
train_embedding = get_embedding_tensor(train_context_ary)

test_context_ary = np.array(x_test)
test_embedding = get_embedding_tensor(test_context_ary)

## Logistic Regression

follow by https://docs.zama.ai/concrete-ml/advanced-topics/prediction_with_fhe


In [ ]:
## https://docs.zama.ai/concrete-ml/advanced-topics/prediction_with_fhe
from concrete.ml.sklearn import LogisticRegression
import time

# Instantiate and train the model
model = LogisticRegression()
model.fit(train_embedding, y_train)

# Simulate the predictions in the clear
start = time.perf_counter()
y_pred_clear = model.predict(test_embedding)
end = time.perf_counter()
print(f"LR computing time: {end - start:.4f} seconds")

# Compile the model on a representative set
start = time.perf_counter()
fhe_circuit = model.compile(train_embedding)
end = time.perf_counter()
print(f"Compilation time: {end - start:.4f} seconds")

# Predict in FHE
start = time.perf_counter()
y_pred_fhe = model.predict(test_embedding, fhe="execute")
end = time.perf_counter()
fhe_exec_time = end - start
print(f"FHE inference time: {fhe_exec_time:.4f} seconds")

LR computing time: 0.1351 seconds
Compilation time: 3.8561 seconds
FHE inference time: 880.2507 seconds


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, classification_report

# accuracy_score
print(f"accuracy_score: {accuracy_score(y_test, y_pred_clear)}")

# precision, recall, f1-score
print(f"Micro precision: {precision_score(y_test, y_pred_clear, average='micro')}")
print(f"Micro recall: {recall_score(y_test, y_pred_clear, average='micro')}")
print(f"Micro f1-score: {f1_score(y_test, y_pred_clear, average='micro')}")

print(f"Macro precision: {precision_score(y_test, y_pred_clear, average='macro')}")
print(f"Macro recall: {recall_score(y_test, y_pred_clear, average='macro')}")
print(f"Macro f1-score: {f1_score(y_test, y_pred_clear, average='macro')}")


# FHE
print()
print("FHE:")
print(f"accuracy_score FHE: {accuracy_score(y_test, y_pred_fhe)}")
print(f"Micro precision FHE: {precision_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro recall FHE: {recall_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro f1-score FHE: {f1_score(y_test, y_pred_fhe, average='micro')}")

print(f"Macro precision FHE: {precision_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro recall FHE: {recall_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro f1-score FHE: {f1_score(y_test, y_pred_fhe, average='macro')}")


# difference between with or not FHE
print()
print("Loss")
print(f"accuracy_score Loss: {accuracy_score(y_test, y_pred_clear) - accuracy_score(y_test, y_pred_fhe)}")
print(f"Micro precision Loss: {precision_score(y_test, y_pred_clear, average='micro') - precision_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro recall Loss: {recall_score(y_test, y_pred_clear, average='micro') - recall_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro f1-score Loss: {f1_score(y_test, y_pred_clear, average='micro') - f1_score(y_test, y_pred_fhe, average='micro')}")
print(f"Macro precision Loss: {precision_score(y_test, y_pred_clear, average='macro') - precision_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro recall Loss: {recall_score(y_test, y_pred_clear, average='macro') - recall_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro f1-score Loss: {f1_score(y_test, y_pred_clear, average='macro') - f1_score(y_test, y_pred_fhe, average='macro')}")

accuracy_score: 0.5881571959638874
Micro precision: 0.5881571959638874
Micro recall: 0.5881571959638874
Micro f1-score: 0.5881571959638874
Macro precision: 0.5772017450645885
Macro recall: 0.5787441474653143
Macro f1-score: 0.5762018842834304

FHE:
accuracy_score FHE: 0.5881571959638874
Micro precision FHE: 0.5881571959638874
Micro recall FHE: 0.5881571959638874
Micro f1-score FHE: 0.5881571959638874
Macro precision FHE: 0.5772017450645885
Macro recall FHE: 0.5787441474653143
Macro f1-score FHE: 0.5762018842834304

Loss
accuracy_score Loss: 0.0
Micro precision Loss: 0.0
Micro recall Loss: 0.0
Micro f1-score Loss: 0.0
Macro precision Loss: 0.0
Macro recall Loss: 0.0
Macro f1-score Loss: 0.0


# By XGBoost

程式運作時間過長

## Classifying with XGBoost

In [ ]:
"""
from concrete.ml.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
# Let's build our model
model = XGBClassifier()
model.fit(train_embedding_tensor, y_train)
# A gridsearch to find the best parameters
parameters = {
    "n_bits": [2, 3],
    "max_depth": [1],
    "n_estimators": [10, 30, 50],
    "n_jobs": [-1],
}

# Now we have a representation for each tweet, we can train a model on these.
grid_search = GridSearchCV(model, parameters, cv=5, n_jobs=5, scoring="accuracy")
grid_search.fit(train_embedding_tensor, y_train)

# Check the accuracy of the best model
print(f"Best score: {grid_search.best_score_}")

# Check best hyperparameters
print(f"Best parameters: {grid_search.best_params_}")

# Extract best model
best_model = grid_search.best_estimator_
"""

'\nfrom concrete.ml.sklearn import XGBClassifier\nfrom sklearn.model_selection import GridSearchCV\n# Let\'s build our model\nmodel = XGBClassifier()\nmodel.fit(train_embedding_tensor, y_train)\n# A gridsearch to find the best parameters\nparameters = {\n    "n_bits": [2, 3],\n    "max_depth": [1],\n    "n_estimators": [10, 30, 50],\n    "n_jobs": [-1],\n}\n\n# Now we have a representation for each tweet, we can train a model on these.\ngrid_search = GridSearchCV(model, parameters, cv=5, n_jobs=5, scoring="accuracy")\ngrid_search.fit(train_embedding_tensor, y_train)\n\n# Check the accuracy of the best model\nprint(f"Best score: {grid_search.best_score_}")\n\n# Check best hyperparameters\nprint(f"Best parameters: {grid_search.best_params_}")\n\n# Extract best model\nbest_model = grid_search.best_estimator_\n'

## Predicting over encrypted data

In [ ]:
"""
import time
# Compute the metrics on the test set
y_pred = best_model.predict(test_embedding_tensor)

# Compile the model to get the FHE inference engine
# (this may take a few minutes depending on the selected model)
start = time.perf_counter()
best_model.compile(train_embedding_tensor)
end = time.perf_counter()
print(f"Compilation time: {end - start:.4f} seconds")

# Now let's predict with FHE over a single tweet and print the time it takes
start = time.perf_counter()
decrypted_predic = best_model.predict(test_embedding_tensor, fhe="execute")
end = time.perf_counter()
fhe_exec_time = end - start
print(f"FHE inference time: {fhe_exec_time:.4f} seconds")

y_pred_HE = decrypted_predic
"""

'\nimport time\n# Compute the metrics on the test set\ny_pred = best_model.predict(test_embedding_tensor)\n\n# Compile the model to get the FHE inference engine\n# (this may take a few minutes depending on the selected model)\nstart = time.perf_counter()\nbest_model.compile(train_embedding_tensor)\nend = time.perf_counter()\nprint(f"Compilation time: {end - start:.4f} seconds")\n\n# Now let\'s predict with FHE over a single tweet and print the time it takes\nstart = time.perf_counter()\ndecrypted_predic = best_model.predict(test_embedding_tensor, fhe="execute")\nend = time.perf_counter()\nfhe_exec_time = end - start\nprint(f"FHE inference time: {fhe_exec_time:.4f} seconds")\n\ny_pred_HE = decrypted_predic\n'

In [ ]:
"""
# accuracy_score
print(f"accuracy_score: {accuracy_score(y_test, y_pred_clear)}")

# precision, recall, f1-score
print(f"Micro precision: {precision_score(y_test, y_pred_clear, average='micro')}")
print(f"Micro recall: {recall_score(y_test, y_pred_clear, average='micro')}")
print(f"Micro f1-score: {f1_score(y_test, y_pred_clear, average='micro')}")

print(f"Macro precision: {precision_score(y_test, y_pred_clear, average='macro')}")
print(f"Macro recall: {recall_score(y_test, y_pred_clear, average='macro')}")
print(f"Macro f1-score: {f1_score(y_test, y_pred_clear, average='macro')}")


# FHE
print()
print("FHE:")
print(f"accuracy_score FHE: {accuracy_score(y_test, y_pred_fhe)}")
print(f"Micro precision FHE: {precision_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro recall FHE: {recall_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro f1-score FHE: {f1_score(y_test, y_pred_fhe, average='micro')}")

print(f"Macro precision FHE: {precision_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro recall FHE: {recall_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro f1-score FHE: {f1_score(y_test, y_pred_fhe, average='macro')}")


# difference between with or not FHE
print()
print("Loss")
print(f"accuracy_score Loss: {accuracy_score(y_test, y_pred_clear) - accuracy_score(y_test, y_pred_fhe)}")
print(f"Micro precision Loss: {precision_score(y_test, y_pred_clear, average='micro') - precision_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro recall Loss: {recall_score(y_test, y_pred_clear, average='micro') - recall_score(y_test, y_pred_fhe, average='micro')}")
print(f"Micro f1-score Loss: {f1_score(y_test, y_pred_clear, average='micro') - f1_score(y_test, y_pred_fhe, average='micro')}")
print(f"Macro precision Loss: {precision_score(y_test, y_pred_clear, average='macro') - precision_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro recall Loss: {recall_score(y_test, y_pred_clear, average='macro') - recall_score(y_test, y_pred_fhe, average='macro')}")
print(f"Macro f1-score Loss: {f1_score(y_test, y_pred_clear, average='macro') - f1_score(y_test, y_pred_fhe, average='macro')}")
"""

'\n# accuracy_score\nprint(f"accuracy_score: {accuracy_score(y_test, y_pred_clear)}")\n\n# precision, recall, f1-score\nprint(f"Micro precision: {precision_score(y_test, y_pred_clear, average=\'micro\')}")\nprint(f"Micro recall: {recall_score(y_test, y_pred_clear, average=\'micro\')}")\nprint(f"Micro f1-score: {f1_score(y_test, y_pred_clear, average=\'micro\')}")\n\nprint(f"Macro precision: {precision_score(y_test, y_pred_clear, average=\'macro\')}")\nprint(f"Macro recall: {recall_score(y_test, y_pred_clear, average=\'macro\')}")\nprint(f"Macro f1-score: {f1_score(y_test, y_pred_clear, average=\'macro\')}")\n\n\n# FHE\nprint()\nprint("FHE:")\nprint(f"accuracy_score FHE: {accuracy_score(y_test, y_pred_fhe)}")\nprint(f"Micro precision FHE: {precision_score(y_test, y_pred_fhe, average=\'micro\')}")\nprint(f"Micro recall FHE: {recall_score(y_test, y_pred_fhe, average=\'micro\')}")\nprint(f"Micro f1-score FHE: {f1_score(y_test, y_pred_fhe, average=\'micro\')}")\n\nprint(f"Macro precision FH